Design RNN or its variant including LSTM or GRU 
Task : 

a) Select a suitable time series dataset.   Example – predict sentiments based on product reviews 

b) Apply for prediction  


In [39]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, GRU, Dense, LSTM
from tensorflow.keras.utils import to_categorical

In [2]:
# Load dataset
df = pd.read_csv("Dataset-SA.csv")
texts = df['Review'].astype(str).values
labels = df['Sentiment'].values

In [59]:
df.head()

,product_name,product_price,Rate,Review,Summary,Sentiment
0,Candes 12 L Room/Personal Air Cooler??????(Whi...,3999,5,super!,great cooler excellent air flow and for this p...,positive
1,Candes 12 L Room/Personal Air Cooler??????(Whi...,3999,5,awesome,best budget 2 fit cooler nice cooling,positive
2,Candes 12 L Room/Personal Air Cooler??????(Whi...,3999,3,fair,the quality is good but the power of air is de...,positive
3,Candes 12 L Room/Personal Air Cooler??????(Whi...,3999,1,useless product,very bad product its a only a fan,negative
4,Candes 12 L Room/Personal Air Cooler??????(Whi...,3999,3,fair,ok ok product,neutral


In [3]:
# Encode sentiment labels
label_encoder = LabelEncoder()
labels_encoded = label_encoder.fit_transform(labels)
labels_categorical = to_categorical(labels_encoded)

In [4]:
# Tokenization and padding
tokenizer = Tokenizer(num_words=5000, oov_token="<OOV>")
tokenizer.fit_on_texts(texts)
sequences = tokenizer.texts_to_sequences(texts)
max_len = max(len(seq) for seq in sequences)
padded_sequences = pad_sequences(sequences, maxlen=max_len, padding='post')

In [5]:
# Train-test split
X_train, X_test, y_train, y_test = train_test_split(
    padded_sequences, labels_categorical, test_size=0.2, random_state=42)

In [41]:
# Build GRU model
model = Sequential([
    Embedding(input_dim=5000, output_dim=64, input_length=max_len),
    LSTM(64),
    Dense(32, activation='relu'),
    Dense(3, activation='softmax')
])

C:\Users\HP\anaconda3\Lib\site-packages\keras\src\layers\core\embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


In [43]:
# Compile
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [45]:
# Train
model.fit(X_train, y_train, epochs=5, batch_size=32, validation_data=(X_test, y_test))

Epoch 1/5
5127/5127 ━━━━━━━━━━━━━━━━━━━━ 43s 8ms/step - accuracy: 0.8414 - loss: 0.4831 - val_accuracy: 0.8994 - val_loss: 0.3485
Epoch 2/5
5127/5127 ━━━━━━━━━━━━━━━━━━━━ 60s 12ms/step - accuracy: 0.8819 - loss: 0.3757 - val_accuracy: 0.8995 - val_loss: 0.3500
Epoch 3/5
5127/5127 ━━━━━━━━━━━━━━━━━━━━ 56s 11ms/step - accuracy: 0.8976 - loss: 0.3469 - val_accuracy: 0.8997 - val_loss: 0.3434
Epoch 4/5
5127/5127 ━━━━━━━━━━━━━━━━━━━━ 58s 11ms/step - accuracy: 0.8987 - loss: 0.3428 - val_accuracy: 0.8998 - val_loss: 0.3425
Epoch 5/5
5127/5127 ━━━━━━━━━━━━━━━━━━━━ 56s 11ms/step - accuracy: 0.8974 - loss: 0.3451 - val_accuracy: 0.8998 - val_loss: 0.3424


In [47]:
# Evaluate
loss, accuracy = model.evaluate(X_test, y_test)
print(f"Test Accuracy: {accuracy*100:.4f}%")

1282/1282 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - accuracy: 0.8980 - loss: 0.3451
Test Accuracy: 89.9783%


In [48]:
def preprocess_review(text, tokenizer, max_len):
    words = text.lower().split()
    sequence = tokenizer.texts_to_sequences([words])
    padded = pad_sequences(sequence, maxlen=max_len, padding='post', truncating='post')
    return padded

In [55]:
def predict_review(review_text):
    sample_padded = preprocess_review(review_text, tokenizer, max_len)
    prediction = model.predict(sample_padded, verbose=0)
    predicted_class = np.argmax(prediction)
    sentiment_label = label_encoder.inverse_transform([predicted_class])[0]
    
    print(f"Predicted Sentiment: {sentiment_label}")

In [57]:
predict_review("The product is amazing and works very well")
predict_review("Worst experience ever, very disappointed")

Predicted Sentiment: positive
Predicted Sentiment: negative
